# Trade network analysis
**Brian Dew (brianwdew@gmail.com)**

`03oil_IFS.ipynb`

Gather additional macroeconomic data from the International Financial Statistics (IFS) series in the IMF API.

Required file:

* region_codes.csv -- maps regions to countries with missing price data

---

   

#### Libraries

In [2]:
import requests                                             # For requesting json data from the url
import pandas as pd                                         # pandas dataframes used for convenience
import os                                                   # change current directory in next line
os.chdir('C:/Working/trade_network/data/')

#### Name local variables

In [3]:
webserv = 'http://dataservices.imf.org/REST/SDMX_JSON.svc/' # the main URL for the JSON rest API
methodCD = 'CompactData/'                                     # CompactData contains only the data 
methodDS = 'DataStructure/' # This method gives info on the country names, units, and indicator names
series = 'IFS'                                              # International Financial Statistics series
freq = 'M'                                                  # Annual
# Set of IFS indicators of interest 
inds = {'x': 'TXG_FOB_USD', 'q': 'EREER_IX', 'mp': 'TMG_D_USD_CIF_IX', 'xp': 'TXG_D_USD_FOB_IX'}  
date = '?startPeriod=2008&endPeriod=2015'                   # Date range of interest
wticall = 'COMMP/M..POILWTI_USD.'

#### Request data on area names, indicator names, and unit multiplier codes

In [10]:
urlDS = webserv+methodDS+series   # url to access IMF datastructure method API
# Request data from IMF JSON RESTful API URL above. Navigate to the code list:
dataDS = requests.get(urlDS).json()['Structure']['CodeLists']['CodeList']#

df = pd.DataFrame(dataDS[2]['Code']).set_index('@value')               # area names here
area_names = {c : df['Description'].loc[c]['#text'] for c in df.index.values}

df = pd.DataFrame(dataDS[3]['Code']).set_index('@value')               # indicator codes
ifs_inds = {i : df['Description'].loc[i]['#text'] for i in df.index.values}

df = pd.DataFrame(dataDS[0]['Code']).set_index('@value')               # unit codes
unit_codes = {m : df['Annotations'].loc[m]['Annotation'][2]['AnnotationText']['#text'] 
              for m in df.index.values}
# A csv file maps countries with missing price data to their regions, and this is read below:
region_codes = pd.read_csv('region_codes.csv', header=None, index_col=0).to_dict()[1] # id'd manually

Request oil price and IFS data

In [11]:
# Obtain oil price data from COMMP series:
url = webserv + methodCD + wticall + date
data = requests.get(url).json()
wti = pd.DataFrame(data['CompactData']['DataSet']['Series']['Obs']).set_index('@TIME_PERIOD')
wti = wti.reset_index().rename(columns={'@TIME_PERIOD': 'date'}).set_index('date')

fd = {} # dictionary for saving each series from inds above
unit_mult = {} # dictionary for saving unit multipliers by country and indicator
for k, v in inds.iteritems(): # k is the key and v is the value
    url = webserv+methodCD+series+'/'+freq+'..'+v+'.'+date # print url to see
    # Build a dataframe for each indicator with the raw data from the IMF API:
    df = pd.DataFrame(requests.get(url).json()
                      ['CompactData']['DataSet']['Series']).set_index('@REF_AREA')
    df['@UNIT_MULT'] = df['@UNIT_MULT'].map(unit_codes) # match unit codes with unit multipliers
    df = df[df['Obs'].apply(lambda x: isinstance(x, list))] # drops empties
    d = {} # temporary dict to save country by country dataframes
    for c in df.index.values: # index values are countries (@REF_AREA) as set above
        d[c] = pd.DataFrame(df.loc[c]['Obs']).rename(columns={'@TIME_PERIOD':'date'})
        # Multiply units by unity multiplier value. 
        d[c]['@OBS_VALUE'] = pd.to_numeric(d[c]['@OBS_VALUE']) * int(df['@UNIT_MULT'][c]) 
    # Concatenate all country rows into one dataframe for each indicator:
    fd[k]= pd.concat(d, axis=0).reset_index().set_index(['level_0','date']).drop('level_1', 1) 

#### Clean up

In [12]:
date_range = fd['xp'].loc['DE'].reset_index().date.values  # Germany selected (automate!)
p_subinds = ['@BASE_YEAR' ,'@OBS_VALUE']       # To save space below var = sub indicators for prices
for k, v in region_codes.iteritems():           # region_codes is a dict: {country code: region code}
    for date in date_range:                     # repeate the replacement for each year in the data.
        fd['mp'].loc[(k, date), p_subinds] = fd['mp'].loc[(v, date), p_subinds]
        fd['xp'].loc[(k, date), p_subinds] = fd['xp'].loc[(v, date), p_subinds] 

#### Merge and save

In [29]:
# The last step is to merge the various indicator values (prices, exports, etc) into one dataframe        
merged = pd.concat(fd, axis=1).reset_index() # combine all series to one merged dataframe
merged['full_name'] = merged['level_0'].map(area_names)    # add column with full name of area
merged = merged.set_index(['level_0','full_name','date'])  # set index to country and date
merged = merged.join(wti)
merged.to_csv('imf_data_oil.csv')                      # drop missing and save as csv file